# Testing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw

In [16]:
def getBoundingBoxes(file_path):
    boundingBoxes = {}
    with open(file_path,'r') as file:
        for line in file:
            l = line.split(',')
            id,x1,y1,width,height,visible = list(map(int,l))
            #bbox = patches.Rectangle((x1, y1), width, height, linewidth=1, edgecolor='red', facecolor='none')
            boundingBoxes[id] = (x1,y1,width,height)
    return boundingBoxes

In [17]:
def draw_bounding_box(image_array, bounding_box, color='yellow', thickness=2):
    # Convert the image array to PIL Image
    image = Image.fromarray(np.uint8(image_array))
    
    # Create a copy of the image
    image_with_box = image.copy()
    
    # Create a draw object
    draw = ImageDraw.Draw(image_with_box)
    
    # Extract the bounding box coordinates
    left, top, width, height = bounding_box
    
    # Calculate the right and bottom coordinates
    right = left + width
    bottom = top + height
    
    # Draw the bounding box
    draw.rectangle([left, top, right, bottom], outline=color, width=thickness)
    
    # Convert the image with the bounding box back to NumPy array
    image_array_with_box = np.array(image_with_box)
    
    return image_array_with_box

In [97]:
def get_dataset(img_path):
    file_path = os.path.join('datasets',img_path,'img')
    txt_path = os.path.join('datasets',img_path, "groundtruth_rect.txt")
    boundingBoxes = getBoundingBoxes(txt_path)
    x = []
    y = []
    i = 1
    for filename in os.listdir(file_path):
        img_path = os.path.join(file_path,filename)
        img = plt.imread(img_path)
        img = cv2.resize(img, (320,180))
        x.append(img)
        #test_img = draw_bounding_box(img,boundingBoxes[i])
        
        y.append(boundingBoxes[i][0])
        
        i+=1
        
    x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.1)
    return (np.array(x_train),np.array(x_test), np.array(y_train), np.array(y_test))

In [98]:
def get_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 320, 3)))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(4))
    model.add(keras.layers.Dense(1))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [99]:
dataset = 'MotorcycleChase'
x_train,x_test, y_train, y_test = get_dataset(dataset)

In [100]:
model = get_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 178, 318, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 89, 159, 32)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 87, 157, 64)       18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 43, 78, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 214656)            0         
                                                                 
 dense_12 (Dense)            (None, 128)              

In [102]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

In [103]:
%%time
history = model.fit(x_train, y_train,epochs=15, validation_data=(x_test,y_test), verbose=1, callbacks= early_stopping)

Epoch 1/15
17/17 [==============================] - 13s 751ms/step - loss: 6.3211e-05 - accuracy: 0.0000e+00 - val_loss: 6.3498e-05 - val_accuracy: 0.0000e+00
Epoch 2/15
17/17 [==============================] - 13s 753ms/step - loss: 6.3211e-05 - accuracy: 0.0000e+00 - val_loss: 6.3498e-05 - val_accuracy: 0.0000e+00
Epoch 3/15
17/17 [==============================] - 13s 752ms/step - loss: 6.3211e-05 - accuracy: 0.0000e+00 - val_loss: 6.3498e-05 - val_accuracy: 0.0000e+00
Epoch 4/15
17/17 [==============================] - 13s 759ms/step - loss: 6.3211e-05 - accuracy: 0.0000e+00 - val_loss: 6.3498e-05 - val_accuracy: 0.0000e+00
Epoch 5/15
17/17 [==============================] - 13s 755ms/step - loss: 6.3211e-05 - accuracy: 0.0000e+00 - val_loss: 6.3498e-05 - val_accuracy: 0.0000e+00
Epoch 6/15
17/17 [==============================] - 13s 760ms/step - loss: 6.3211e-05 - accuracy: 0.0000e+00 - val_loss: 6.3498e-05 - val_accuracy: 0.0000e+00
Epoch 7/15
17/17 [============================

In [1]:
img_path = './datasets/MotorcycleChase/img/00001.jpg'
img = plt.imread(img_path)
img = cv2.resize(img, (320,180))
prediction = model.predict(img)
prob = prediction.max()

NameError: name 'plt' is not defined

In [106]:
img.shape

(180, 320, 3)